# Most Polluted Countries - Limpieza de Datos

### ¬∑ Antes de empezar... 

1. Para gestionar los Dataframes:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from collections import Counter
from fuzzywuzzy import process, fuzz
import pickle

/Users/paulaelizagarate/opt/anaconda3/envs/Bootcamp/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#Regex
import re

###  ¬∑ Carga de CSVs

Empezamos cargando datos CSV procedentes de Kaggle con la informaci√≥n de los pa√≠ses y las ciudades m√°s contaminadas desde 2017 hasta 2021. 

In [4]:
df_pcountries = pd.read_csv("datos/AIR QUALITY INDEX- top countries.csv", index_col = 0 )
df_pcountries.head(2)

Country/Region  2021  2020  2019  2018   Population
Rank                                                    
1        Bangladesh  76.9  77.1  83.3  97.1  164,689,383
2              Chad  75.9     -     -     -   16,425,859

In [5]:
#df_pcountries.values

In [6]:
df_pcities = pd.read_csv("datos/AIR QUALITY INDEX (by cities) - IQAir.csv", index_col = 0 )
df_pcities.head(2)

City   2021 JAN(2021) FEB(2021) MAR(2021) APR(2021)  \
Rank                                                                    
1       Bhiwadi, India  106.2     145.8     129.8     120.2     125.7   
2     Ghaziabad, India  102.0     199.9     172.2      97.8      86.3   

     MAY(2021) JUN(2021) JUL(2021) AUG(2021) SEP(2021) OCT(2021) NOV(2021)  \
Rank                                                                         
1         86.5      95.9      55.6      55.4      37.1      91.1     188.6   
2         52.9      47.2      35.3      37.6      30.8      89.7     218.3   

     DEC(2021)   2020   2019   2018   2017  
Rank                                        
1        136.6   95.5   83.4  125.4      -  
2          163  106.6  110.2  135.2  144.6

A continuaci√≥n, cargamos nuestros CSVs con los datos scrapeados acerca de la natalidad en el mismo periodo. 

In [7]:
df_n17= pd.read_csv("datos/natalidad/natalidad2017.csv" ,  index_col = 0)
df_n17.head(5)

0
1  Espa√±a [+] 2017 393.181 202.478 190.703 8,41‚Ä∞ ...
2  Alemania [+] 2017 784.901 402.517 382.384 9,50...
3  Reino Unido [+] 2017 754.754 387.030 367.724 1...
4  Francia [+] 2017 770.045 394.058 375.987 11,50...
5  Italia [+] 2017 458.151 235.733 222.418 7,60‚Ä∞ ...

In [8]:
df_n18= pd.read_csv("datos/natalidad/natalidad2018.csv" ,  index_col = 0)
df_n18.head(5)

0
1  Espa√±a [+] 2018 372.777 191.569 181.208 7,94‚Ä∞ ...
2  Alemania [+] 2018 787.523 404.052 383.471 9,50...
3  Reino Unido [+] 2018 730.918 375.379 355.539 1...
4  Francia [+] 2018 759.199 388.944 370.255 11,30...
5  Italia [+] 2018 439.747 226.217 213.530 7,30‚Ä∞ ...

In [9]:
df_n19= pd.read_csv("datos/natalidad/natalidad2019.csv" ,  index_col = 0)
df_n19.head(5)

0
1  Espa√±a [+] 2019 360.617 185.523 175.094 7,62‚Ä∞ ...
2  Alemania [+] 2019 778.090 399.292 378.798 9,40...
3  Reino Unido [+] 2019 712.699 375.379 355.539 1...
4  Francia [+] 2019 754.008 385.340 368.668 11,20...
5  Italia [+] 2019 420.084 215.387 204.697 7,00‚Ä∞ ...

In [10]:
df_n20= pd.read_csv("datos/natalidad/natalidad2020.csv" ,  index_col = 0)
df_n20.head(5)

0
1  Espa√±a [+] 2020 341.315 174.842 166.473 7,19‚Ä∞ ...
2  Alemania [+] 2020 773.144 397.385 375.759 9,30...
3  Reino Unido [+] 2020 712.699 375.379 355.539 1...
4  Francia [+] 2020 735.775 376.187 359.588 10,90...
5  Italia [+] 2020 404.892 208.431 196.461 6,80‚Ä∞ ...

In [11]:
df_n21= pd.read_csv("datos/natalidad/natalidad2021.csv" ,  index_col = 0)
df_n21.head(5)

0
1  Espa√±a [+] 2021 337.380 174.148 163.232 7,12‚Ä∞ ...
2  Alemania [+] 2021 795.492 397.385 375.759 9,60...
3  Reino Unido [+] 2020 712.699 375.379 355.539 1...
4  Francia [+] 2021 738.594 376.187 359.588 10,90...
5  Italia [+] 2021 399.431 208.431 196.461 6,80‚Ä∞ ...

### ¬∑ Limpieza Datos Scrapeados 

In [12]:
df_n17

0
1    Espa√±a [+] 2017 393.181 202.478 190.703 8,41‚Ä∞ ...
2    Alemania [+] 2017 784.901 402.517 382.384 9,50...
3    Reino Unido [+] 2017 754.754 387.030 367.724 1...
4    Francia [+] 2017 770.045 394.058 375.987 11,50...
5    Italia [+] 2017 458.151 235.733 222.418 7,60‚Ä∞ ...
..                                                 ...
189                  Samoa [+] 2017 24,69‚Ä∞ 3,93 -1,26%
190                  Yemen [+] 2017 31,04‚Ä∞ 3,89 -2,60%
191              Sud√°frica [+] 2017 20,91‚Ä∞ 2,43 -1,10%
192                 Zambia [+] 2017 36,70‚Ä∞ 4,72 -1,99%
193               Zimbabue [+] 2017 31,73‚Ä∞ 3,71 -2,55%

[193 rows x 1 columns]

Empezaremos por limpiar el grueso de nuestros datos, discriminando los pa√≠ses. La decisi√≥n de hacerlo de esta manera es porque el split de la mayor√≠a de informaci√≥n lo haremos por espacios y en los pa√≠ses no es v√°lido ya que muchos tienen nombre compuesto.

In [13]:
df_n17= df_n17["0"].str.split("\s\[", regex= True, expand=True)
df_n17

0                                                  1
1         Espa√±a  +] 2017 393.181 202.478 190.703 8,41‚Ä∞ 1,31 -1,92%
2       Alemania  +] 2017 784.901 402.517 382.384 9,50‚Ä∞ 1,57 -1,88%
3    Reino Unido  +] 2017 754.754 387.030 367.724 11,40‚Ä∞ 1,74 -2...
4        Francia  +] 2017 770.045 394.058 375.987 11,50‚Ä∞ 1,89 -1...
5         Italia  +] 2017 458.151 235.733 222.418 7,60‚Ä∞ 1,32 -1,49%
..           ...                                                ...
189        Samoa                         +] 2017 24,69‚Ä∞ 3,93 -1,26%
190        Yemen                         +] 2017 31,04‚Ä∞ 3,89 -2,60%
191    Sud√°frica                         +] 2017 20,91‚Ä∞ 2,43 -1,10%
192       Zambia                         +] 2017 36,70‚Ä∞ 4,72 -1,99%
193     Zimbabue                         +] 2017 31,73‚Ä∞ 3,71 -2,55%

[193 rows x 2 columns]

Para hacer la divisi√≥n del grueso de datos, crearemos un diccionario. Aprovecharemos que la estructura de los datos extra√≠dos es fija para realizar el almacenamiento en el mismo. Lo primero es conocer la longitud de los diferentes paquetes de informaci√≥n, para crear los condicionales de los que depender√° este almacenamiento. En aquellos datos que no est√©n informados, con el objetivo de mantener la estructura, introduciremos un valor nulo. Este diccionario ser√° nuestro puente para los DataFrames finales. 

In [14]:
lon=[]

for i in df_n17[1]:
    lon.append(len(i.split()))

In [15]:
lon[:5]

[8, 8, 8, 8, 8]

In [16]:
numeros_unicos = set(lon)

print(numeros_unicos)

{8, 5, 6, 7}


In [17]:
#lon.index(7)

Una vez sabemos la longitud de nuestros distintos paquetes de datos, empezamos con la creaci√≥n del diccionario:

In [18]:
nat2017 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n17[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(str(a[2]))
        nat2017["male_born"].append(str(a[3]))
        nat2017["female_born"].append(str(a[4]))
        nat2017["birth_rate"].append(str(a[5]))
        nat2017["fertility_rate"].append(str(a[6]))
        nat2017["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(str(a[2]))
        nat2017["male_born"].append(str(a[3]))
        nat2017["female_born"].append(str(a[4]))
        nat2017["birth_rate"].append(str(a[5]))
        nat2017["fertility_rate"].append(str(a[6]))
        nat2017["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(str(a[2]))
        nat2017["male_born"].append(np.nan)
        nat2017["female_born"].append(np.nan)
        nat2017["birth_rate"].append(str(a[3]))
        nat2017["fertility_rate"].append(str(a[4]))
        nat2017["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(np.nan)
        nat2017["male_born"].append(np.nan)
        nat2017["female_born"].append(np.nan)
        nat2017["birth_rate"].append(str(a[2]))
        nat2017["fertility_rate"].append(str(a[3]))
        nat2017["variation"].append(str(a[4]))
            

In [19]:
#control
len(nat2017["year"])

193

Una vez finalizado nuestro diccionario con el grueso de datos, debemos hacer la limpieza de la columna pa√≠ses. La limpieza se basar√° en eliminar espacios o "y" en los nombres, en su lugar pondremos "_": 

In [20]:
df_n17[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [21]:
df_n17[0] = df_n17[0].apply(lambda x: str(x).replace(" y ","_"))
df_n17[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [22]:
df_n17[0] = df_n17[0].apply(lambda x: str(x).replace(" ","_"))
df_n17[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

Ya disponemos de los pa√≠ses limpios por lo que est√°n listos para ser unidos al resto del diccionario. Esto nos formar√° uno de nuestros cinco DataFrames finales. Haremos un diccionario con los pa√≠ses limpios, que despu√©s zipearemos con la informaci√≥n de nuestro anterior diccionario:

In [23]:
paises17={
        "country": []
            }

for i in df_n17[0]:
    
    paises17["country"].append(i)

In [24]:
#paises17

In [25]:
union17 = list(zip(paises17["country"], nat2017["year"], nat2017["born"], nat2017["male_born"], nat2017["female_born"], nat2017["birth_rate"], nat2017["fertility_rate"] , nat2017["variation"]))

###### *PUNTO DE CONTROL* üìç
Necesario para reseteo DataFrame

In [34]:
n17= pd.DataFrame(union17)
n17

0     1        2        3        4       5     6       7
0         Espa√±a  2017  393.181  202.478  190.703   8,41‚Ä∞  1,31  -1,92%
1       Alemania  2017  784.901  402.517  382.384   9,50‚Ä∞  1,57  -1,88%
2    Reino_Unido  2017  754.754  387.030  367.724  11,40‚Ä∞  1,74  -2,79%
3        Francia  2017  770.045  394.058  375.987  11,50‚Ä∞  1,89  -1,56%
4         Italia  2017  458.151  235.733  222.418   7,60‚Ä∞  1,32  -1,49%
..           ...   ...      ...      ...      ...     ...   ...     ...
188        Samoa  2017      NaN      NaN      NaN  24,69‚Ä∞  3,93  -1,26%
189        Yemen  2017      NaN      NaN      NaN  31,04‚Ä∞  3,89  -2,60%
190    Sud√°frica  2017      NaN      NaN      NaN  20,91‚Ä∞  2,43  -1,10%
191       Zambia  2017      NaN      NaN      NaN  36,70‚Ä∞  4,72  -1,99%
192     Zimbabue  2017      NaN      NaN      NaN  31,73‚Ä∞  3,71  -2,55%

[193 rows x 8 columns]

In [27]:
#control
n17.info()
n17.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       193 non-null    object
 1   1       193 non-null    object
 2   2       55 non-null     object
 3   3       51 non-null     object
 4   4       51 non-null     object
 5   5       193 non-null    object
 6   6       193 non-null    object
 7   7       192 non-null    object
dtypes: object(8)
memory usage: 12.2+ KB


0      0
1      0
2    138
3    142
4    142
5      0
6      0
7      1
dtype: int64

Nuestro siguiente paso ser√° dar nombre a las columnas:

In [28]:
n17.columns

RangeIndex(start=0, stop=8, step=1)

In [35]:
n17.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1064/526978396.py:1: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  n17.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


El siguiente paso ser√° gestionar los tipos de datos que tenemos. Nos interesa tener la mayor cantidad de datos num√©ricos posibles:

In [36]:
n17["year"]= n17["year"].astype("int")

In [32]:
n17["born"].fillna("", inplace= True)

In [37]:
n17.dtypes

country           object
year               int64
born              object
male_born         object
female_born       object
birth_rate        object
fertility_rate    object
variation         object
dtype: object

In [40]:
n17["born"] = n17["born"].str.replace(".", "")

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1064/2687544233.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  n17["born"] = n17["born"].str.replace(".", "")


In [41]:
n17.dtypes

country           object
year               int64
born              object
male_born         object
female_born       object
birth_rate        object
fertility_rate    object
variation         object
dtype: object

In [43]:
n17["born"] = pd.to_numeric(n17["born"])

In [44]:
n17.dtypes

country            object
year                int64
born              float64
male_born          object
female_born        object
birth_rate         object
fertility_rate     object
variation          object
dtype: object

In [33]:
n17["born"].astype("float")

ValueError: could not convert string to float: '2.234.039'

In [213]:
n17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   country         193 non-null    object
 1   year            193 non-null    int64 
 2   born            193 non-null    object
 3   male_born       51 non-null     object
 4   female_born     51 non-null     object
 5   birth_rate      193 non-null    object
 6   fertility_rate  193 non-null    object
 7   variation       192 non-null    object
dtypes: int64(1), object(7)
memory usage: 12.2+ KB


In [198]:
n17

country  year     born male_born female_born birth_rate  \
0         Espa√±a  2017  393.181   202.478     190.703      8,41‚Ä∞   
1       Alemania  2017  784.901   402.517     382.384      9,50‚Ä∞   
2    Reino_Unido  2017  754.754   387.030     367.724     11,40‚Ä∞   
3        Francia  2017  770.045   394.058     375.987     11,50‚Ä∞   
4         Italia  2017  458.151   235.733     222.418      7,60‚Ä∞   
..           ...   ...      ...       ...         ...        ...   
188        Samoa  2017                NaN         NaN     24,69‚Ä∞   
189        Yemen  2017                NaN         NaN     31,04‚Ä∞   
190    Sud√°frica  2017                NaN         NaN     20,91‚Ä∞   
191       Zambia  2017                NaN         NaN     36,70‚Ä∞   
192     Zimbabue  2017                NaN         NaN     31,73‚Ä∞   

    fertility_rate variation  
0             1,31    -1,92%  
1             1,57    -1,88%  
2             1,74    -2,79%  
3             1,89    -1,56%  
4             1,32    -1,49%  
..             ...       ...  
188           3,93    -1,26%  
189           3,89    -2,60%  
190           2,43    -1,10%  
191           4,72    -1,99%  
192           3,71    -2,55%  

[193 rows x 8 columns]

#### ¬°Lo tenemos! ‚ú®

Primer DataFrame limpio y listo para ser guardado para nuestra fase de anal√≠sis. 

In [138]:
n17.to_csv('n17_def.csv')

Ahora a por los otros 4 a√±os de informaci√≥n, donde repetiremos el proceso realizado:

### 2018

In [216]:
df_n18

0
1    Espa√±a [+] 2018 372.777 191.569 181.208 7,94‚Ä∞ ...
2    Alemania [+] 2018 787.523 404.052 383.471 9,50...
3    Reino Unido [+] 2018 730.918 375.379 355.539 1...
4    Francia [+] 2018 759.199 388.944 370.255 11,30...
5    Italia [+] 2018 439.747 226.217 213.530 7,30‚Ä∞ ...
..                                                 ...
189                  Samoa [+] 2018 24,38‚Ä∞ 3,88 -1,25%
190                  Yemen [+] 2018 30,45‚Ä∞ 3,79 -2,52%
191              Sud√°frica [+] 2018 20,51‚Ä∞ 2,41 -1,03%
192                 Zambia [+] 2018 36,19‚Ä∞ 4,63 -1,80%
193               Zimbabue [+] 2018 30,68‚Ä∞ 3,62 -2,48%

[193 rows x 1 columns]

In [217]:
df_n18= df_n18["0"].str.split("\s\[", regex= True, expand=True)
df_n18

0                                                  1
1         Espa√±a  +] 2018 372.777 191.569 181.208 7,94‚Ä∞ 1,26 -3,67%
2       Alemania       +] 2018 787.523 404.052 383.471 9,50‚Ä∞ 1,57 0
3    Reino Unido  +] 2018 730.918 375.379 355.539 11,00‚Ä∞ 1,68 -3...
4        Francia  +] 2018 759.199 388.944 370.255 11,30‚Ä∞ 1,87 -1...
5         Italia  +] 2018 439.747 226.217 213.530 7,30‚Ä∞ 1,29 -2,27%
..           ...                                                ...
189        Samoa                         +] 2018 24,38‚Ä∞ 3,88 -1,25%
190        Yemen                         +] 2018 30,45‚Ä∞ 3,79 -2,52%
191    Sud√°frica                         +] 2018 20,51‚Ä∞ 2,41 -1,03%
192       Zambia                         +] 2018 36,19‚Ä∞ 4,63 -1,80%
193     Zimbabue                         +] 2018 30,68‚Ä∞ 3,62 -2,48%

[193 rows x 2 columns]

In [222]:
lon18=[]

for i in df_n18[1]:
    lon18.append(len(i.split()))

In [223]:
unicos18 = set(lon18)

print(unicos18)

{8, 5, 6, 7}


In [224]:
nat2018 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n18[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(str(a[2]))
        nat2018["male_born"].append(str(a[3]))
        nat2018["female_born"].append(str(a[4]))
        nat2018["birth_rate"].append(str(a[5]))
        nat2018["fertility_rate"].append(str(a[6]))
        nat2018["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(str(a[2]))
        nat2018["male_born"].append(str(a[3]))
        nat2018["female_born"].append(str(a[4]))
        nat2018["birth_rate"].append(str(a[5]))
        nat2018["fertility_rate"].append(str(a[6]))
        nat2018["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(str(a[2]))
        nat2018["male_born"].append(np.nan)
        nat2018["female_born"].append(np.nan)
        nat2018["birth_rate"].append(str(a[3]))
        nat2018["fertility_rate"].append(str(a[4]))
        nat2018["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(np.nan)
        nat2018["male_born"].append(np.nan)
        nat2018["female_born"].append(np.nan)
        nat2018["birth_rate"].append(str(a[2]))
        nat2018["fertility_rate"].append(str(a[3]))
        nat2018["variation"].append(str(a[4]))

In [225]:
#control
len(nat2018["year"])

193

In [226]:
df_n18[0] = df_n18[0].apply(lambda x: str(x).replace(" y ","_"))
df_n18[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [227]:
df_n18[0] = df_n18[0].apply(lambda x: str(x).replace(" ","_"))
df_n18[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [228]:
paises18={
        "country": []
            }

for i in df_n18[0]:
    
    paises18["country"].append(i)

In [235]:
union18 = list(zip(paises18["country"], nat2018["year"], nat2018["born"], nat2018["male_born"], nat2018["female_born"], nat2018["birth_rate"], nat2018["fertility_rate"] , nat2018["variation"]))

In [236]:
n18= pd.DataFrame(union18)
n18

0     1        2        3        4       5     6       7
0         Espa√±a  2018  372.777  191.569  181.208   7,94‚Ä∞  1,26  -3,67%
1       Alemania  2018  787.523  404.052  383.471   9,50‚Ä∞  1,57       0
2    Reino_Unido  2018  730.918  375.379  355.539  11,00‚Ä∞  1,68  -3,45%
3        Francia  2018  759.199  388.944  370.255  11,30‚Ä∞  1,87  -1,06%
4         Italia  2018  439.747  226.217  213.530   7,30‚Ä∞  1,29  -2,27%
..           ...   ...      ...      ...      ...     ...   ...     ...
188        Samoa  2018      NaN      NaN      NaN  24,38‚Ä∞  3,88  -1,25%
189        Yemen  2018      NaN      NaN      NaN  30,45‚Ä∞  3,79  -2,52%
190    Sud√°frica  2018      NaN      NaN      NaN  20,51‚Ä∞  2,41  -1,03%
191       Zambia  2018      NaN      NaN      NaN  36,19‚Ä∞  4,63  -1,80%
192     Zimbabue  2018      NaN      NaN      NaN  30,68‚Ä∞  3,62  -2,48%

[193 rows x 8 columns]

### 2019

In [232]:
df_n19

0
1    Espa√±a [+] 2019 360.617 185.523 175.094 7,62‚Ä∞ ...
2    Alemania [+] 2019 778.090 399.292 378.798 9,40...
3    Reino Unido [+] 2019 712.699 375.379 355.539 1...
4    Francia [+] 2019 754.008 385.340 368.668 11,20...
5    Italia [+] 2019 420.084 215.387 204.697 7,00‚Ä∞ ...
..                                                 ...
189                  Samoa [+] 2019 24,10‚Ä∞ 3,83 -1,21%
190                  Yemen [+] 2019 29,87‚Ä∞ 3,70 -2,43%
191              Sud√°frica [+] 2019 20,13‚Ä∞ 2,38 -1,00%
192                 Zambia [+] 2019 35,78‚Ä∞ 4,56 -1,60%
193               Zimbabue [+] 2019 29,75‚Ä∞ 3,53 -2,32%

[193 rows x 1 columns]

In [233]:
df_n19= df_n19["0"].str.split("\s\[", regex= True, expand=True)
df_n19

0                                                  1
1         Espa√±a  +] 2019 360.617 185.523 175.094 7,62‚Ä∞ 1,24 -2,24%
2       Alemania  +] 2019 778.090 399.292 378.798 9,40‚Ä∞ 1,54 -1,91%
3    Reino Unido  +] 2019 712.699 375.379 355.539 10,70‚Ä∞ 1,63 -2...
4        Francia  +] 2019 754.008 385.340 368.668 11,20‚Ä∞ 1,86 -0...
5         Italia  +] 2019 420.084 215.387 204.697 7,00‚Ä∞ 1,27 -1,55%
..           ...                                                ...
189        Samoa                         +] 2019 24,10‚Ä∞ 3,83 -1,21%
190        Yemen                         +] 2019 29,87‚Ä∞ 3,70 -2,43%
191    Sud√°frica                         +] 2019 20,13‚Ä∞ 2,38 -1,00%
192       Zambia                         +] 2019 35,78‚Ä∞ 4,56 -1,60%
193     Zimbabue                         +] 2019 29,75‚Ä∞ 3,53 -2,32%

[193 rows x 2 columns]

In [234]:
lon19=[]

for i in df_n19[1]:
    lon19.append(len(i.split()))

In [237]:
unicos19 = set(lon19)

print(unicos19)

{8, 5, 6, 7}


In [238]:
nat2019 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n19[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(str(a[2]))
        nat2019["male_born"].append(str(a[3]))
        nat2019["female_born"].append(str(a[4]))
        nat2019["birth_rate"].append(str(a[5]))
        nat2019["fertility_rate"].append(str(a[6]))
        nat2019["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(str(a[2]))
        nat2019["male_born"].append(str(a[3]))
        nat2019["female_born"].append(str(a[4]))
        nat2019["birth_rate"].append(str(a[5]))
        nat2019["fertility_rate"].append(str(a[6]))
        nat2019["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(str(a[2]))
        nat2019["male_born"].append(np.nan)
        nat2019["female_born"].append(np.nan)
        nat2019["birth_rate"].append(str(a[3]))
        nat2019["fertility_rate"].append(str(a[4]))
        nat2019["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(np.nan)
        nat2019["male_born"].append(np.nan)
        nat2019["female_born"].append(np.nan)
        nat2019["birth_rate"].append(str(a[2]))
        nat2019["fertility_rate"].append(str(a[3]))
        nat2019["variation"].append(str(a[4]))

In [239]:
#control
len(nat2019["year"])

193

In [240]:
df_n19[0] = df_n19[0].apply(lambda x: str(x).replace(" y ","_"))
df_n19[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [241]:
df_n19[0] = df_n19[0].apply(lambda x: str(x).replace(" ","_"))
df_n19[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [242]:
paises19={
        "country": []
            }

for i in df_n19[0]:
    
    paises19["country"].append(i)

In [259]:
union19 = list(zip(paises19["country"], nat2019["year"], nat2019["born"], nat2019["male_born"], nat2019["female_born"], nat2019["birth_rate"], nat2019["fertility_rate"] , nat2019["variation"]))

In [260]:
n19= pd.DataFrame(union19)
n19

0     1        2        3        4       5     6       7
0         Espa√±a  2019  360.617  185.523  175.094   7,62‚Ä∞  1,24  -2,24%
1       Alemania  2019  778.090  399.292  378.798   9,40‚Ä∞  1,54  -1,91%
2    Reino_Unido  2019  712.699  375.379  355.539  10,70‚Ä∞  1,63  -2,98%
3        Francia  2019  754.008  385.340  368.668  11,20‚Ä∞  1,86  -0,53%
4         Italia  2019  420.084  215.387  204.697   7,00‚Ä∞  1,27  -1,55%
..           ...   ...      ...      ...      ...     ...   ...     ...
188        Samoa  2019      NaN      NaN      NaN  24,10‚Ä∞  3,83  -1,21%
189        Yemen  2019      NaN      NaN      NaN  29,87‚Ä∞  3,70  -2,43%
190    Sud√°frica  2019      NaN      NaN      NaN  20,13‚Ä∞  2,38  -1,00%
191       Zambia  2019      NaN      NaN      NaN  35,78‚Ä∞  4,56  -1,60%
192     Zimbabue  2019      NaN      NaN      NaN  29,75‚Ä∞  3,53  -2,32%

[193 rows x 8 columns]

### 2020

In [245]:
df_n20

0
1    Espa√±a [+] 2020 341.315 174.842 166.473 7,19‚Ä∞ ...
2    Alemania [+] 2020 773.144 397.385 375.759 9,30...
3    Reino Unido [+] 2020 712.699 375.379 355.539 1...
4    Francia [+] 2020 735.775 376.187 359.588 10,90...
5    Italia [+] 2020 404.892 208.431 196.461 6,80‚Ä∞ ...
..                                                 ...
189                  Samoa [+] 2020 23,86‚Ä∞ 3,79 -1,17%
190                  Yemen [+] 2020 29,30‚Ä∞ 3,61 -2,32%
191              Sud√°frica [+] 2020 19,77‚Ä∞ 2,36 -0,97%
192                 Zambia [+] 2020 35,44‚Ä∞ 4,50 -1,38%
193               Zimbabue [+] 2020 28,98‚Ä∞ 3,46 -2,01%

[193 rows x 1 columns]

In [246]:
df_n20= df_n20["0"].str.split("\s\[", regex= True, expand=True)
df_n20

0                                                  1
1         Espa√±a  +] 2020 341.315 174.842 166.473 7,19‚Ä∞ 1,19 -4,03%
2       Alemania  +] 2020 773.144 397.385 375.759 9,30‚Ä∞ 1,53 -0,65%
3    Reino Unido  +] 2020 712.699 375.379 355.539 10,20‚Ä∞ 1,56 -4...
4        Francia  +] 2020 735.775 376.187 359.588 10,90‚Ä∞ 1,83 -1...
5         Italia  +] 2020 404.892 208.431 196.461 6,80‚Ä∞ 1,24 -2,36%
..           ...                                                ...
189        Samoa                         +] 2020 23,86‚Ä∞ 3,79 -1,17%
190        Yemen                         +] 2020 29,30‚Ä∞ 3,61 -2,32%
191    Sud√°frica                         +] 2020 19,77‚Ä∞ 2,36 -0,97%
192       Zambia                         +] 2020 35,44‚Ä∞ 4,50 -1,38%
193     Zimbabue                         +] 2020 28,98‚Ä∞ 3,46 -2,01%

[193 rows x 2 columns]

In [247]:
lon20=[]

for i in df_n20[1]:
    lon20.append(len(i.split()))

In [248]:
unicos20 = set(lon20)

print(unicos20)

{8, 5, 6, 7}


In [249]:
nat2020 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n20[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(str(a[2]))
        nat2020["male_born"].append(str(a[3]))
        nat2020["female_born"].append(str(a[4]))
        nat2020["birth_rate"].append(str(a[5]))
        nat2020["fertility_rate"].append(str(a[6]))
        nat2020["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(str(a[2]))
        nat2020["male_born"].append(str(a[3]))
        nat2020["female_born"].append(str(a[4]))
        nat2020["birth_rate"].append(str(a[5]))
        nat2020["fertility_rate"].append(str(a[6]))
        nat2020["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(str(a[2]))
        nat2020["male_born"].append(np.nan)
        nat2020["female_born"].append(np.nan)
        nat2020["birth_rate"].append(str(a[3]))
        nat2020["fertility_rate"].append(str(a[4]))
        nat2020["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(np.nan)
        nat2020["male_born"].append(np.nan)
        nat2020["female_born"].append(np.nan)
        nat2020["birth_rate"].append(str(a[2]))
        nat2020["fertility_rate"].append(str(a[3]))
        nat2020["variation"].append(str(a[4]))

In [250]:
#control
len(nat2020["year"])

193

In [252]:
df_n20[0] = df_n20[0].apply(lambda x: str(x).replace(" y ","_"))
df_n20[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [251]:
df_n20[0] = df_n20[0].apply(lambda x: str(x).replace(" ","_"))
df_n20[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [253]:
paises20={
        "country": []
            }

for i in df_n20[0]:
    
    paises20["country"].append(i)

In [256]:
union20 = list(zip(paises20["country"], nat2020["year"], nat2020["born"], nat2020["male_born"], nat2020["female_born"], nat2020["birth_rate"], nat2020["fertility_rate"] , nat2020["variation"]))

In [258]:
n20= pd.DataFrame(union20)
n20

0     1        2        3        4       5     6       7
0         Espa√±a  2020  341.315  174.842  166.473   7,19‚Ä∞  1,19  -4,03%
1       Alemania  2020  773.144  397.385  375.759   9,30‚Ä∞  1,53  -0,65%
2    Reino_Unido  2020  712.699  375.379  355.539  10,20‚Ä∞  1,56  -4,29%
3        Francia  2020  735.775  376.187  359.588  10,90‚Ä∞  1,83  -1,61%
4         Italia  2020  404.892  208.431  196.461   6,80‚Ä∞  1,24  -2,36%
..           ...   ...      ...      ...      ...     ...   ...     ...
188        Samoa  2020      NaN      NaN      NaN  23,86‚Ä∞  3,79  -1,17%
189        Yemen  2020      NaN      NaN      NaN  29,30‚Ä∞  3,61  -2,32%
190    Sud√°frica  2020      NaN      NaN      NaN  19,77‚Ä∞  2,36  -0,97%
191       Zambia  2020      NaN      NaN      NaN  35,44‚Ä∞  4,50  -1,38%
192     Zimbabue  2020      NaN      NaN      NaN  28,98‚Ä∞  3,46  -2,01%

[193 rows x 8 columns]

### 2021

In [232]:
df_n21

0
1    Espa√±a [+] 2019 360.617 185.523 175.094 7,62‚Ä∞ ...
2    Alemania [+] 2019 778.090 399.292 378.798 9,40...
3    Reino Unido [+] 2019 712.699 375.379 355.539 1...
4    Francia [+] 2019 754.008 385.340 368.668 11,20...
5    Italia [+] 2019 420.084 215.387 204.697 7,00‚Ä∞ ...
..                                                 ...
189                  Samoa [+] 2019 24,10‚Ä∞ 3,83 -1,21%
190                  Yemen [+] 2019 29,87‚Ä∞ 3,70 -2,43%
191              Sud√°frica [+] 2019 20,13‚Ä∞ 2,38 -1,00%
192                 Zambia [+] 2019 35,78‚Ä∞ 4,56 -1,60%
193               Zimbabue [+] 2019 29,75‚Ä∞ 3,53 -2,32%

[193 rows x 1 columns]

In [261]:
df_n21= df_n21["0"].str.split("\s\[", regex= True, expand=True)
df_n21

0                                                  1
1         Espa√±a   +] 2021 337.380 174.148 163.232 7,12‚Ä∞ 1,19 0,47%
2       Alemania  +] 2021 795.492 397.385 375.759 9,60‚Ä∞ 1,53 -0,65%
3    Reino Unido  +] 2020 712.699 375.379 355.539 10,20‚Ä∞ 1,56 -4...
4        Francia  +] 2021 738.594 376.187 359.588 10,90‚Ä∞ 1,83 -1...
5         Italia  +] 2021 399.431 208.431 196.461 6,80‚Ä∞ 1,24 -2,36%
..           ...                                                ...
189        Samoa                         +] 2020 23,86‚Ä∞ 3,79 -1,17%
190        Yemen                         +] 2020 29,30‚Ä∞ 3,61 -2,32%
191    Sud√°frica                         +] 2020 19,77‚Ä∞ 2,36 -0,97%
192       Zambia                         +] 2020 35,44‚Ä∞ 4,50 -1,38%
193     Zimbabue                         +] 2020 28,98‚Ä∞ 3,46 -2,01%

[193 rows x 2 columns]

In [262]:
lon21=[]

for i in df_n21[1]:
    lon21.append(len(i.split()))

In [263]:
unicos21 = set(lon21)

print(unicos21)

{8, 5, 6, 7}


In [264]:
nat2021 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n21[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(str(a[2]))
        nat2021["male_born"].append(str(a[3]))
        nat2021["female_born"].append(str(a[4]))
        nat2021["birth_rate"].append(str(a[5]))
        nat2021["fertility_rate"].append(str(a[6]))
        nat2021["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(str(a[2]))
        nat2021["male_born"].append(str(a[3]))
        nat2021["female_born"].append(str(a[4]))
        nat2021["birth_rate"].append(str(a[5]))
        nat2021["fertility_rate"].append(str(a[6]))
        nat2021["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(str(a[2]))
        nat2021["male_born"].append(np.nan)
        nat2021["female_born"].append(np.nan)
        nat2021["birth_rate"].append(str(a[3]))
        nat2021["fertility_rate"].append(str(a[4]))
        nat2021["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(np.nan)
        nat2021["male_born"].append(np.nan)
        nat2021["female_born"].append(np.nan)
        nat2021["birth_rate"].append(str(a[2]))
        nat2021["fertility_rate"].append(str(a[3]))
        nat2021["variation"].append(str(a[4]))

In [265]:
#control
len(nat2021["year"])

193

In [266]:
df_n21[0] = df_n21[0].apply(lambda x: str(x).replace(" y ","_"))
df_n21[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [267]:
df_n21[0] = df_n21[0].apply(lambda x: str(x).replace(" ","_"))
df_n21[0].unique()[:5]

array(['Espa√±a', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [268]:
paises21={
        "country": []
            }

for i in df_n21[0]:
    
    paises21["country"].append(i)

In [269]:
union21 = list(zip(paises21["country"], nat2021["year"], nat2021["born"], nat2021["male_born"], nat2021["female_born"], nat2021["birth_rate"], nat2021["fertility_rate"] , nat2021["variation"]))

In [271]:
n21= pd.DataFrame(union21)
n21

0     1        2        3        4       5     6       7
0         Espa√±a  2021  337.380  174.148  163.232   7,12‚Ä∞  1,19   0,47%
1       Alemania  2021  795.492  397.385  375.759   9,60‚Ä∞  1,53  -0,65%
2    Reino_Unido  2020  712.699  375.379  355.539  10,20‚Ä∞  1,56  -4,29%
3        Francia  2021  738.594  376.187  359.588  10,90‚Ä∞  1,83  -1,61%
4         Italia  2021  399.431  208.431  196.461   6,80‚Ä∞  1,24  -2,36%
..           ...   ...      ...      ...      ...     ...   ...     ...
188        Samoa  2020      NaN      NaN      NaN  23,86‚Ä∞  3,79  -1,17%
189        Yemen  2020      NaN      NaN      NaN  29,30‚Ä∞  3,61  -2,32%
190    Sud√°frica  2020      NaN      NaN      NaN  19,77‚Ä∞  2,36  -0,97%
191       Zambia  2020      NaN      NaN      NaN  35,44‚Ä∞  4,50  -1,38%
192     Zimbabue  2020      NaN      NaN      NaN  28,98‚Ä∞  3,46  -2,01%

[193 rows x 8 columns]